<img src="https://images.efollett.com/htmlroot/images/templates/storeLogos/CA/864.gif" style="float: right;"> 




# ECON611
### HW_4 -  SQL
#### Due: Nov 18

We will be using the sample databases that come from the DBeaver program (use the Database Installation guide to assist you on installing it). We will be using the sample tables that are provided there. Save your work as a SQL script with answers to specific questions commented out properly labled.

## 1. Music and Album Data
---
We will be using the following tables from DBeaver:
• Album • Artist • Genre • Track • PlaylistTrack • MediaType

## Problems (40 points)
---

#### A. (5 points). Output a table that has the columns Album Title, Artist, Track Name, Genre, Composer, Length of Song (minutes) 
---


select 
album.Title as Album_Title, 
artist.Name as Artist, 
track.name as Track_Name, 
genre.name as Genre, 
track.Composer as Composer,
track.Milliseconds/(1000*60) as Length_in_Minutes
from album 
join artist on album.artistid = artist.artistid
join track on track.albumid = album.AlbumId
join genre on genre.GenreId = track.GenreId

/%reference: https://javarevisited.blogspot.com/2012/11/how-to-join-three-tables-in-sql-query-mysql-sqlserver.html %/


# B. (5 points). Output a frequency table for how many letters are in an album title and a song title. (i.e. 3 columns: number of letters, number of album titles with that amount of letters, number of song titles with that amount of letters). What is the average and range of amount of letters?
---

select x.charct, x.hist as album_hist, y.hist as track_hist
from
(
select charct, count(charct) as hist from 
(select album.title, length(album.Title) as charct from album) group by charct
) as x
left JOIN
(
select charct, count(charct) as hist from 
(select track.Name, length(track.Name) as charct from track) group by charct
) as y
on x.charct = y.charct
/*where charct = number of characters
album_hist = count of albums with charct
track_hist = count of tracks with charct*/

/*TKTK: unsure how to capture NULLS for both album and track; i.e. 0 albums may have x characters, 0 tracks may have y characters*/
/*reference: https://www.essentialsql.com/get-ready-to-learn-sql-server-22-using-subqueries-in-the-from-clause/ 
https://www.geeksengine.com/database/subquery/subquery-in-join-operation.php */

#### C. (10 points). How many songs did each composer write? How many artists composed their own music? Output all of the Artists who had songs that were longer than 2.5 minutes and they did not write their own songs.
---

/*songs written per composer*/
select track.Composer, count(track.Composer)
from Track group by track.Composer

/*count of self-written artists*/
select count (DISTINCT composer) as 'Count of self-composed artists:' from (select track.TrackId, track.composer, artist.Name
from track join album on track.AlbumId = album.AlbumId
join artist on album.ArtistId = artist.ArtistId
where track.composer is artist.name) 

/*artists with songs longer than 2.5min, not self-written*/
select distinct name from (select track.TrackId, track.composer, track.milliseconds/(1000) as time_in_sec, artist.Name
from track join album on track.AlbumId = album.AlbumId
join artist on album.ArtistId = artist.ArtistId
where track.composer is not artist.name
and time_in_sec > 150)

#### D. (10 points). How many albums contain songs that only have an even number of seconds in the length of the song and the songs are all MPEG audio files?
---

select count (distinct albumid) from (select track.albumid, track.milliseconds/1000 as length_in_seconds, track.MediaTypeId from track
where (length_in_seconds%2)=0
and MediaTypeId=1)

#### E. (10 points). How large, in terms of Megabytes are each album? On average, how large, in terms of bytes, is a pop song? What is the range and average duration of a heavy metal song in minutes?
---

/*mb size of each album */
select y.album_name, x.mb_length_by_album
from 
	(select track.albumid as album_id, sum(track.Bytes)/(power(10,6)) as mb_length_by_album 
	from track group by track.AlbumId) as x
left JOIN
	(select album.AlbumId as album_id, album.Title as album_name from album) as y
on x.album_id = y.album_id

/*average pop song byte size*/
select avg(bytes) from track where GenreId=9

/*range, avg duration of heavy metal song in minutes*/
select (MAX(length_in_min) - MIN(length_in_min)) as duration_range, avg(length_in_min)
from(select track.TrackId, track.Milliseconds/60000 as length_in_min from track 
where track.GenreId= 13)

## 2. Customer Data
---
We will be using the following tables from DBeaver:
• Customer • Invoice • InvoiceLine • Album • Artist • Genre • Track • PlaylistTrack • MediaType

## Problems (40 points)
---

#### A. (5 points). Output a table that shows the number of customers in each city, and also have shows the state and country that the city is located in.
---

select count(customer.CustomerId) as ct_customers, customer.city, customer.state, customer.country 
from customer 
group by customer.city

## B. (10 points). How much in total was spent on music per year? What is the average amount a person spent on music in a year? What is the average amount a person with a gmail account spent on music in a year?
---

/*INCOMPLETE*/
/*the following returns sales from 2007*/
select sum(total) as sum_2007 
from Invoice 
where invoicedate like '%2007%'
/*reference: https://sqlhints.com/tag/sql-group-by-year/ */

/*Average spent per person in a year*/
select sum(sum_per_cust)/count(customerid)/4 as avg_per_person_per_year 
from (select invoice.customerid, customer.email, sum(invoice.total) as sum_per_cust 
from invoice 
join customer on invoice.CustomerId=customer.CustomerId 
group by invoice.CustomerId)

#### C. (10 points). What year-month combination had the most spending on music? Which year-month combi- nation had the most money spent on pop music?
---

/*Incomplete*/

#### D. (10 points). What artist has the most songs bought that are in the pop genre? What is the total money spent on these songs? Which customer is spending the most on pop songs?
---

/*artist with most pop songs bought*/
select max(sales) as highest_pop_sales, name as highest_pop_artist from (
select count(ArtistId) as sales, name from (
select track.trackid, album.artistid, artist.name, invoiceline.Quantity
from album 
join invoiceline on invoiceline.TrackId=track.TrackId
join artist on artist.ArtistId = album.artistid
join track on track.albumid = album.albumid where track.genreid=9) group by name) 

/*total money spent on that artist's (U2) songs*/
select round(sum(quantity*UnitPrice), 2) as revenue 
from (
select track.trackid, invoiceline.Quantity, invoiceline.UnitPrice from track
join invoiceline on track.trackid = invoiceline.TrackId
join album on track.albumid = album.AlbumId
where album.ArtistId=150)

/*highest pop song customer*/
select max(total) as highest_pop_spending, customerid, firstname, lastname from (select invoice.customerid, customer.FirstName, customer.LastName, invoice.total
from invoiceline
join track on track.trackid = invoiceline.TrackId
join invoice 
join customer on customer.CustomerId=invoice.CustomerId
where track.GenreId = 9
group by invoice.CustomerId) 

#### E. (10 points). Output how many songs each artist is selling in each country? Who is selling the post outside of the US? On average, how many songs is each artist selling outside of the US?
---

/*INCOMPLETE*/

/*the following shows number of songs bought per country*/
select sum(invoiceline.Quantity), invoice.BillingCountry
from invoiceline join invoice on invoiceline.InvoiceId = invoice.InvoiceId
group by invoice.BillingCountry


## 3. Window Functions (20 points)
---

#### A. (10 points). Return a list of Customers that assigns a sequential integer to each customer and resets the number when the country of the customer changes for Customers are not from Canada and Brazil and whose email address have `yahoo` or `gmail'
---

#### B. (10 points). Compute the rank for each track in each album based on the track’s length, then append all the tracks that have a lengthrank of 2 and 4 
---

select * from (SELECT
	albumid,
	name,
	milliseconds,
	DENSE_RANK () over(
		PARTITION by albumid
		order by milliseconds) as lengthrank
from track) where lengthrank=4 or lengthrank=2

## BONUS (20 points)
---
- Using SQLITE syntax [make your own SQLITE table](https://www.sqlitetutorial.net/sqlite-create-table/) and set of data. You will need to create a table of 5 columns and 10 rows containing any information you find interesting. (5 points)
- You should create/import your table need to [dbeaver](https://docs.roguewave.com/en/zendstudio/current/content/create_manage_database_with_dbeaver.htm). (5 points)
- Perform 3 different calculations using your data in order to tell a story of your data set. (10 points)

** FEEL FREE TO USE/EXPLORE OTHER SOURCES**